In [1]:
import numpy as np     
import pandas as pd                       
import matplotlib.pyplot as plt

from pykat import finesse                 
from pykat.commands import *               
from pykat.optics.maps import * 
from pykat.parallel import parakat

%config InlineBackend.figure_format='svg'
%matplotlib inline
pykat.init_pykat_plotting(dpi=90)

                                              ..-
    PyKat 1.2.1           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



## Content:
This file is about calculating the circulating mode contents for LG22 and HG33 mode as input with zernike maps applied. The selection rule for LG22 is checked.

In [3]:
#clipping loss of 1ppm
katHG33 = pykat.finesse.kat()
katHG33.verbose=False
katHG33.parse("""
l laser 1 0 0 n0
s s0 1 n0 nmod1

mod eom 60M 0.3 1 pm nmod1 nmod2
s s1 0 nmod2 n1

m1 ITM 0.014 3.75e-05 0 n1 n2
s sCav 3994.4692 n2 n3
m1 ETM 5e-06 3.75e-05 3.8 n3 n4

#higher order mode TEM33
tem laser 0 0 0.0 0.0
tem laser 3 3 1.0 0.0

#Gaussian model
#Rc=2013.35 so that the loss is 1ppm
cav cavity ITM n2 ETM n3
attr ITM Rc -2679.93
attr ETM Rc 2679.93
#Rc=2679.93 when clipping loss is 1ppm for HG33

pd1 pdh 60M 0 n1

set err pdh re
lock clock $err 1 1n
put* ETM phi $clock


maxtem 8

""")

In [4]:
#clipping loss of 1ppm
katLG22 = pykat.finesse.kat()
katLG22.verbose=False
katLG22.parse("""

l laser 1 0 0 n0
#gauss g1 laser n0 1m 0
maxtem 8

s s0 1 n0 nmod1
mod eom 60M 0.3 1 pm nmod1 nmod2
s s1 0 nmod2 n1
m1 ITM 0.014 3.75e-05 0 n1 n2
s sCav 3994.4692 n2 n3
m1 ETM 5e-06 3.75e-05 26.5 n3 n4

#higher order mode LG22
tem laser 0 0 0 0
tem laser 6 0 0.234375 0
tem laser 5 1 0.15625 -90
tem laser 4 2 0.015625 0
tem laser 3 3 0.1875 -90
tem laser 2 4 0.015625 180
tem laser 1 5 0.15625 -90
tem laser 0 6 0.234375 180

#Gaussian model
cav cavity ITM n2 ETM n3
attr ITM Rc -2789.58
attr ETM Rc 2789.58
#Rc=2789.58 when clipping loss is 1ppm for LG22

pd1 pdh 60M 0 n1
set err pdh re
lock clock $err 1 1n
put* ETM phi $clock

""")

In [5]:
# Calculate the mode content besides LG22 to check the selection rule
dic={}
i=0
pkmodecheck=parakat()
for n in range(2,7,2):
    for m in range(-n,n+1,2):
        dic[f'{i}']=f'Z{n}{m}' # Conversion between 'nm' basis to 'i' to rename the column names of the dataframe
        i=i+1
        LGmodecheck = katLG22.deepcopy()
        LGmodecheck.parse(f"""
                map ETM Znmmaps/Z{n}{m}order1phasemap.txt
                ad AD60 6 0 0 n3
                ad AD51 5 1 0 n3
                ad AD42 4 2 0 n3
                ad AD33 3 3 0 n3
                ad AD24 2 4 0 n3
                ad AD15 1 5 0 n3
                ad AD06 0 6 0 n3
                pd Pcirc n3
                yaxis abs:deg
                """)
        LGmodecheck.noxaxis = True
        pkmodecheck.run(LGmodecheck)
out = pkmodecheck.getResults()

dfLGmode=pd.DataFrame(index=['LG0-6', 'LG1-4', 'LG2-2', 'LG30', 'LG22', 'LG14', 'LG06'])
dfLGmodeprob=pd.DataFrame(index=['LG0-6', 'LG1-4', 'LG2-2', 'LG30', 'LG22', 'LG14', 'LG06'])
for i in range(15):
    LGmode=[]
    LGmodeprob=[]
    for l in range(-6,7,2):
        mode=0
        for n in range(7):
            for m in range(6,-1,-1):
                if n+m==6:
                    coeff,p,L = pykat.optics.gaussian_beams.HG2LG(n,m)
                    mode += coeff[np.where(L == l)]*out[i][f"AD{n}{m}"] # calculate each LG mode from all of the HG 
                                                                # modes of order 6 and the conversion coefficients(coeff)
        modepow=(np.abs(mode)**2)[0]
        LGmode.append(modepow)
        LGmodeprob.append(100*modepow/np.real(out[i]["Pcirc"])) # relative power
    dfLGmode[f'{i}']=LGmode
    dfLGmodeprob[f'{i}']=LGmodeprob
dfLGmode.rename(columns=dic, inplace=True) # use the zernike index as the new column names
dfLGmodeprob.rename(columns=dic, inplace=True)

dfLGmode.to_pickle('pkl/dfLGmode.pkl')
dfLGmodeprob.to_pickle('pkl/dfLGmodeprob.pkl')
# In the order of LGpl: LG0-6, LG1-4, LG2-2, LG30, LG22, LG14, LG06

Parallel jobs: 100%|##########################################################|

In [6]:
dfLGmode=pd.read_pickle('pkl/dfLGmode.pkl')
dfLGmodeprob=pd.read_pickle('pkl/dfLGmodeprob.pkl')

## Selection rule: $$ l^\prime = l\pm m=2\pm m  $$

In [7]:
dfLGmode

,Z2-2,Z20,Z22,Z4-4,Z4-2,Z40,Z42,Z44,Z6-6,Z6-4,Z6-2,Z60,Z62,Z64,Z66
LG0-6,0.001084,4.380344e-04,0.000991,1.909567e-03,0.007363,4.229760e-04,0.007220,7.180635e-04,0.000338,1.184197e-01,0.002444,3.525048e-04,0.000979,9.218543e-02,4.144111e-04
LG1-4,0.008376,1.964265e-11,0.009436,3.218907e-07,0.093332,3.310842e-11,0.100566,1.308622e-09,0.161367,4.534306e-06,0.007566,5.451159e-11,0.012295,5.582385e-09,1.475190e-01
LG2-2,0.280619,8.350098e-05,0.297421,1.155594e+00,1.316504,8.237012e-05,1.348801,1.186288e+00,0.000109,7.461067e+00,0.316869,8.527025e-05,0.380606,7.760568e+00,7.854599e-05
LG30,8.276755,5.030235e-11,8.245210,6.727747e-05,16.127381,3.234937e-08,16.120510,2.188866e-08,0.000002,2.132065e-05,9.042733,6.630819e-08,9.244819,1.025141e-08,2.460116e-07
LG22,235.371958,2.677204e+02,235.335182,2.645061e+02,197.559896,2.676326e+02,197.300386,2.643586e+02,267.402382,2.456557e+02,233.407458,2.673465e+02,232.932882,2.449395e+02,2.674308e+02
LG14,7.047419,2.002736e-11,7.074043,4.288688e-06,14.373545,2.489324e-08,14.439011,1.825086e-08,0.000232,7.547340e-07,7.163000,5.075324e-08,7.101675,7.375078e-09,4.405223e-07
LG06,0.143651,3.668996e-04,0.119911,4.976492e-01,0.715624,3.531171e-04,0.668932,4.982231e-01,0.000071,3.150876e+00,0.098857,2.976014e-04,0.089857,3.331308e+00,3.432245e-04


## $HG_{33}$:
The selection rule(if any) for the HG33 case is left unknown

In [11]:
# HG33 mode pseudodegenaracy
dic={}
i=0
pkmodecheck=parakat()
for n in range(8):
    for m in range(-n,n+1,2):
        dic[f'{i}']=f'Z{n}{m}' # use for column names of the dataframe later
        i=i+1
        HGmodecheck = katHG33.deepcopy()
        HGmodecheck.parse(f"""
                map ETM Znmmaps/Z{n}{m}order1phasemap.txt
                ad AD60 6 0 0 n3
                ad AD51 5 1 0 n3
                ad AD42 4 2 0 n3
                ad AD33 3 3 0 n3
                ad AD24 2 4 0 n3
                ad AD15 1 5 0 n3
                ad AD06 0 6 0 n3
                pd Pcirc n3
                yaxis abs
                """)
        HGmodecheck.noxaxis = True
        pkmodecheck.run(HGmodecheck)
out = pkmodecheck.getResults()

dfHGmode=pd.DataFrame(index=['HG06', 'HG15', 'HG24', 'HG33', 'HG42', 'HG51', 'HG60'])
dfHGmodeprob=pd.DataFrame(index=['HG06', 'HG15', 'HG24', 'HG33', 'HG42', 'HG51', 'HG60'])
for i in range(36):
    HGmode=[]
    HGmodeprob=[]
    for n in range(7):
        for m in range(6,-1,-1):
            if n+m==6:
                HGmode.append(out[i][f"AD{n}{m}"]**2)
                HGmodeprob.append(100*out[i][f"AD{n}{m}"]**2/out[i]["Pcirc"])
    
    dfHGmode[f'{i}']=HGmode
    dfHGmodeprob[f'{i}']=HGmodeprob
dfHGmode.rename(columns=dic, inplace=True) # use the zernike index as the new column names
dfHGmodeprob.rename(columns=dic, inplace=True)

dfHGmode.to_pickle('pkl/dfHGall.pkl')

Parallel jobs: 100%|##########################################################|

In [12]:
pd.options.display.max_columns = 40
dfHGall=pd.read_pickle('pkl/dfHGall.pkl')
dfHGall                  

,Z00,Z1-1,Z11,Z2-2,Z20,Z22,Z3-3,Z3-1,Z31,Z33,Z4-4,Z4-2,Z40,Z42,Z44,Z5-5,Z5-3,Z5-1,Z51,Z53,Z55,Z6-6,Z6-4,Z6-2,Z60,Z62,Z64,Z66,Z7-7,Z7-5,Z7-3,Z7-1,Z71,Z73,Z75,Z77
HG06,5.158687e-40,6.654469e-35,1.117233e-34,0.003914,9.295716e-14,1.064769e-14,7.463171e-08,6.633412e-08,1.650827e-08,1.646367e-08,0.000643,0.052031,5.540093e-10,8.256176e-14,3.666673e-11,2.828265e-04,2.101786e-09,2.264815e-08,1.735903e-08,4.217226e-08,5.460991e-09,3.385068e-03,0.001338,0.001115,5.025176e-09,6.950990e-11,3.010183e-10,6.856144e-11,2.240540e-08,2.424381e-09,5.649168e-08,8.284495e-08,1.003258e-09,2.486803e-10,7.940894e-09,1.577444e-08
HG15,2.067003e-08,2.033337e-08,2.034001e-08,0.278354,1.532048e-08,2.100814e-08,6.243328e-06,5.583222e-05,6.073174e-06,1.216738e-06,0.000255,1.176368,1.126547e+00,2.353933e-08,2.853596e-01,2.188721e-05,2.210573e-04,2.920254e-03,5.923836e-04,2.949949e-04,2.682671e-05,9.440736e-07,0.013407,0.220511,6.735303e+00,3.325215e-02,1.832820e+00,3.411310e-02,5.198089e-06,1.026005e-04,2.494252e-03,2.576253e-03,1.839472e-05,6.585689e-04,6.988857e-05,5.206788e-06
HG24,2.280823e-30,1.078525e-37,4.929484e-31,8.673274,6.765345e-12,1.412062e-13,2.975232e-04,9.072828e-05,2.036348e-05,2.030131e-05,0.112592,16.102409,5.230953e-08,6.428353e-12,2.915378e-08,6.744046e-06,8.956129e-04,7.374010e-04,3.013642e-07,1.744749e-06,6.035576e-07,8.949484e-02,0.733245,6.968541,4.711818e-08,4.256753e-09,7.888641e-09,5.449060e-09,1.984977e-05,1.753622e-05,3.764638e-04,1.935707e-04,3.609731e-05,4.502320e-05,2.060609e-05,1.530873e-05
HG33,2.680935e+02,2.680905e+02,2.680905e+02,231.041393,2.680919e+02,2.680931e+02,2.680505e+02,2.680137e+02,2.680215e+02,2.680554e+02,267.626132,192.864410,2.635248e+02,2.680793e+02,2.669470e+02,2.680853e+02,2.678448e+02,2.677775e+02,2.678121e+02,2.678772e+02,2.680863e+02,2.677220e+02,265.056922,238.502872,2.402769e+02,2.678804e+02,2.606855e+02,2.679552e+02,2.680913e+02,2.680246e+02,2.676111e+02,2.677484e+02,2.677986e+02,2.676881e+02,2.680308e+02,2.680915e+02
HG42,1.303108e-38,3.443292e-31,3.141378e-33,8.672361,5.305179e-12,1.228014e-13,3.217379e-04,6.421683e-04,1.726539e-05,1.721594e-05,0.116882,16.099636,3.653636e-08,6.863574e-12,4.516166e-08,1.389980e-05,5.506558e-04,7.177866e-04,4.656809e-07,5.585617e-07,2.062487e-06,8.961198e-02,0.729693,6.954308,1.611422e-08,3.379871e-09,1.457571e-09,4.162832e-09,3.463634e-05,1.200217e-04,3.977538e-06,1.224242e-05,2.967031e-05,1.941622e-05,5.143772e-06,1.133688e-05
HG51,2.238683e-08,2.148966e-08,2.182310e-08,0.278268,1.678716e-08,2.232221e-08,1.043017e-06,6.193555e-06,5.562167e-05,7.599163e-06,0.000282,1.176962,1.126591e+00,2.572216e-08,2.853646e-01,2.539393e-05,9.787206e-05,6.528364e-04,2.931218e-03,5.923195e-04,2.210930e-05,6.707671e-07,0.013686,0.220838,6.735399e+00,3.317662e-02,1.832899e+00,3.413081e-02,5.262340e-06,8.013863e-05,2.065138e-03,2.200040e-04,1.511100e-03,5.460448e-04,1.171392e-04,5.370743e-06
HG60,1.806964e-35,1.569890e-36,9.206987e-35,0.003775,8.378633e-13,8.691848e-15,2.423807e-05,2.376220e-08,1.060157e-08,1.051946e-08,0.000009,0.050799,2.163619e-10,2.780030e-13,8.941309e-11,6.319709e-09,9.770042e-09,1.612622e-09,1.235919e-08,1.209161e-09,5.981146e-09,2.912059e-03,0.000693,0.000085,4.098169e-10,4.327530e-12,6.635467e-11,5.488800e-12,2.115438e-07,6.402951e-10,4.165563e-09,5.497527e-10,1.652589e-10,2.491159e-09,1.713194e-08,6.627848e-09
